# Car residual value modelling (PoC)

## I. Importer les librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import joblib
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.models import DataPreprocessor, univariate_analysis, bivariate_analysis, multivariate_analysis, scatter_3d
from src.models import create_transformer_pipeline
from src.models import CategoricalEmbedding, plot_embeddings, visualize_all_embeddings, get_embedding_weights, cluster_embeddings
from src.models import model_evaluation
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
import tensorflow as tf
import xgboost as xgb

## II. Data Preprocessing

### 1. Déclaration des chemins de data

In [2]:
# Le chemin vers les données d'annonce
annonce_path = os.path.join("..", "data", "raw_data", "autohero.csv")

# Le chemin vers les données de prix neuf
prix_neuf_path = os.path.join("..", "data", "scraping_prix_neuf", "prix_neuf_voitures_vf.csv")

### 2. Création d'une instance de Data Preprocessor

In [3]:
# Créer une instance de la classe DataPreprocessor
preprocessor = DataPreprocessor(file_path=annonce_path)

### 3. Charger les données d'annonces & les prétransformer

In [4]:
# Charger les données
preprocessor.load_data()
# Afficher la dimension du DataFrame
print(f"La dimension du DataFrame est: {preprocessor.data.shape}") 
preprocessor.data.head()

Data loaded successfully from ..\data\raw_data\autohero.csv
La dimension du DataFrame est: (2352, 20)


,scraped_at,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,classe_vehicule,nb_porte,nb_place,couleur,sellerie,classe_emission,emission_CO2,crit_air,usage_commerciale_anterieure,url_annonce
0,2025-04-09,Ford Fiesta,1.0 EcoBoost ST-Line X,13 190 €,15.05.2020,69 301 km,Essence,Boite de vitesse manuelle,95 CV / 70 kW,3,Citadine,5.0,5.0,Gris,Tissu (Sellerie d'origine),EURO 6,NaN,Crit'Air 1,NaN,https://www.autohero.com/fr/ford-fiesta/id/516...
1,2025-04-09,Toyota ProAce,Combi Long 1.5 D-4D Dynamic,23 990 €,29.04.2021,71 887 km,Diesel,Boite de vitesse manuelle,120 CV / 88 kW,2,Monospace,4.0,9.0,Gris,Tissu (Sellerie d'origine),EURO 6,170 g/km,Crit'Air 2,Oui,https://www.autohero.com/fr/toyota-pro-ace/id/...
2,2025-04-09,Mercedes-Benz GLA,250 e AMG Line 8G-DCT,32 490 €,23.10.2020,59 649 km,Hybride,Double embrayage / DCT,218 CV / 160 kW,2,SUV,5.0,5.0,Gris,Mi-cuir (Sellerie d'origine),EURO 6,32 g/km,Crit'Air 1,Oui,https://www.autohero.com/fr/mercedes-benz-gla/...
3,2025-04-09,BMW X1,sDrive18i xLine DKG7,27 890 €,05.05.2021,37 869 km,Essence,Double embrayage / DCT,136 CV / 100 kW,2,SUV,5.0,5.0,Noir,Mi-cuir (Sellerie d'origine),EURO 6,148 g/km,Crit'Air 1,Non,https://www.autohero.com/fr/bmw-x-1/id/490f203...
4,2025-04-09,Peugeot 3008,1.5 Blue-HDi Crossway EAT8,19 090 €,31.12.2019,58 958 km,Diesel,Boite de vitesse automatique,130 CV / 96 kW,3,SUV,5.0,5.0,Blanc,Mi-cuir (Sellerie d'origine),EURO 6,98 g/km,Crit'Air 2,Oui,https://www.autohero.com/fr/peugeot-3008/id/df...


In [5]:
# Prétransformer les données en ajoutant les nouvelles colonnes et homogénéisant les données
preprocessor.pretransform_data()
print(f"Après la pré-transformation, la nouvelle dimension est: {preprocessor.data.shape}")
preprocessor.data.head() 

Après la pré-transformation, la nouvelle dimension est: (2352, 30)


,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,url_annonce,annee,age_days,age_years,age_months,km_per_year,km_per_month,modele_alt,finition_puissance,id_annonce
0,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,https://www.autohero.com/fr/ford-fiesta/id/516...,2020,1790.0,4.9,59.7,14143.1,1160.8,FORD FIESTA,1.0 ECOBOOST ST-LINE X 95 CV,1
1,2025-04-09,TOYOTA,TOYOTA PROACE,COMBI LONG 1.5 D-4D DYNAMIC,23990.0,29.04.2021,71887.0,Diesel,Boite de vitesse manuelle,120,...,https://www.autohero.com/fr/toyota-pro-ace/id/...,2021,1441.0,3.9,48.0,18432.6,1497.6,TOYOTA PROACE,COMBI LONG 1.5 D-4D DYNAMIC 120 CV,2
2,2025-04-09,MERCEDES,MERCEDES-BENZ GLA,250 E AMG LINE 8G-DCT,32490.0,23.10.2020,59649.0,Hybride,Boite de vitesse automatique,218,...,https://www.autohero.com/fr/mercedes-benz-gla/...,2020,1629.0,4.5,54.3,13255.3,1098.5,MERCEDES-BENZ GLA,250 E AMG LINE 8G-DCT 218 CV,3
3,2025-04-09,BMW,BMW X1,SDRIVE18I XLINE DKG7,27890.0,05.05.2021,37869.0,Essence,Boite de vitesse automatique,136,...,https://www.autohero.com/fr/bmw-x-1/id/490f203...,2021,1435.0,3.9,47.8,9710.0,792.2,BMW X1,SDRIVE18I XLINE DKG7 136 CV,4
4,2025-04-09,PEUGEOT,PEUGEOT 3008,1.5 BLUE-HDI CROSSWAY EAT8,19090.0,31.12.2019,58958.0,Diesel,Boite de vitesse automatique,130,...,https://www.autohero.com/fr/peugeot-3008/id/df...,2019,1926.0,5.3,64.2,11124.2,918.3,PEUGEOT 3008,1.5 BLUE-HDI CROSSWAY EAT8 130 CV,5


### 4. Récupération des prix neuf

In [6]:
# Récupérer le prix neuf
preprocessor.merge_new_price(prix_neuf_path)
print(f"Après la récupération du prix neuf, la nouvelle dimension est: {preprocessor.data.shape}") 
preprocessor.data.head()

Après la récupération du prix neuf, la nouvelle dimension est: (6812, 52)


,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,np_energie,np_prix_neuf,np_version_finale,note_version_commune,note_transmission_commune,note_carburant_commun,note_nb_porte_commun,note_totale_commune,max_note,nb_match_par_annonce
0,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Essence,21650,FORD FIESTA 6 VI 1.0 ECOBOOST 95 S/S ST-LINE X 3P,5,1,1,0,7,7,2
1,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Essence,21650,FORD FIESTA 6 VI 1.0 ECOBOOST 95 S/S ST-LINE X 5P,5,1,1,0,7,7,2
2,2025-04-09,TOYOTA,TOYOTA PROACE,COMBI LONG 1.5 D-4D DYNAMIC,23990.0,29.04.2021,71887.0,Diesel,Boite de vitesse manuelle,120,...,Diesel,34290,TOYOTA PROACE 2 II (2) COMBI LONG 1.5 120 D-4D...,6,1,1,0,8,8,2
3,2025-04-09,TOYOTA,TOYOTA PROACE,COMBI LONG 1.5 D-4D DYNAMIC,23990.0,29.04.2021,71887.0,Diesel,Boite de vitesse manuelle,120,...,Diesel,34240,TOYOTA PROACE 2 II (2) PROACE COMBI LONG 1.5 1...,6,1,1,0,8,8,2
4,2025-04-09,MERCEDES,MERCEDES-BENZ GLA,250 E AMG LINE 8G-DCT,32490.0,23.10.2020,59649.0,Hybride,Boite de vitesse automatique,218,...,Hybride,55600,MERCEDES GLA 2 II 250 E AMG LINE 8G-DCT,5,1,1,0,7,7,1


In [7]:
print(f"Nombre total d'annonces: {preprocessor.data['id_annonce'].drop_duplicates().shape[0]}")
print(f"Nombre d'annonces où le prix neuf est renseigné: {preprocessor.data[preprocessor.data['np_prix_neuf'].notnull()][['id_annonce']].drop_duplicates().shape[0]}")
print(f"Nombre d'annonces où le prix neuf n'est pas trouvé: {preprocessor.data[preprocessor.data['np_prix_neuf'].isnull()][['id_annonce']].drop_duplicates().shape[0]}")

Nombre total d'annonces: 2352
Nombre d'annonces où le prix neuf est renseigné: 2339
Nombre d'annonces où le prix neuf n'est pas trouvé: 13


In [8]:
# Annonces dont le prix neuf est manquant
df = preprocessor.data.copy()
df_miss_prix = df[df["np_prix_neuf"].isnull()]
df_miss_prix.value_counts(subset=['modele', 'modele_alt'])

modele                    modele_alt
CITROEN C4 GRAND PICASSO  C4 PICASSO    1
CITROEN C4 PICASSO        C4 PICASSO    1
KIA PRO_CEE'D             PROCEED       1
Name: count, dtype: int64

In [9]:
df_miss_prix

,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,np_energie,np_prix_neuf,np_version_finale,note_version_commune,note_transmission_commune,note_carburant_commun,note_nb_porte_commun,note_totale_commune,max_note,nb_match_par_annonce
1077,2025-04-09,CITROEN,CITROEN C4 PICASSO,(2) 1.2 PURETECH SHINE EAT6,11690.0,24.04.2019,65767.0,Essence,Boite de vitesse automatique,130,...,NaN,NaN,NaN,0,0,0,0,0,0,1
2105,2025-04-09,CITROEN,CITROEN C4 GRAND PICASSO,1.2 PURETECH SHINE BV6,10590.0,31.05.2019,87702.0,Essence,Boite de vitesse manuelle,130,...,NaN,NaN,NaN,0,0,0,0,0,0,1
2108,2025-04-09,KIA,KIA PRO_CEE'D,1.0 T-GDI ISG GT-LINE BV6,10790.0,16.03.2017,84553.0,Essence,Boite de vitesse manuelle,120,...,NaN,NaN,NaN,0,0,0,0,0,0,1
2364,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,...,NaN,NaN,NaN,0,0,0,0,0,0,1
2823,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,...,NaN,NaN,NaN,0,0,0,0,0,0,1
3015,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,...,NaN,NaN,NaN,0,0,0,0,0,0,1
4441,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,...,NaN,NaN,NaN,0,0,0,0,0,0,1
5086,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,...,NaN,NaN,NaN,0,0,0,0,0,0,1
5408,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,...,NaN,NaN,NaN,0,0,0,0,0,0,1
5638,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,...,NaN,NaN,NaN,0,0,0,0,0,0,1


13 sur 2352 lignes où le prix neuf est introuvable, dont 10 annonces vides.  
Vu le nombre non significatif de manquants => les exclure de la base de travail

In [10]:
preprocessor.data = preprocessor.data[preprocessor.data['np_prix_neuf'].notnull()]
print(f"Nombre total d'annonces restant: {preprocessor.data['id_annonce'].drop_duplicates().shape[0]}")

Nombre total d'annonces restant: 2339


In [11]:
preprocessor.data.head()

,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,np_energie,np_prix_neuf,np_version_finale,note_version_commune,note_transmission_commune,note_carburant_commun,note_nb_porte_commun,note_totale_commune,max_note,nb_match_par_annonce
0,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Essence,21650,FORD FIESTA 6 VI 1.0 ECOBOOST 95 S/S ST-LINE X 3P,5,1,1,0,7,7,2
1,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Essence,21650,FORD FIESTA 6 VI 1.0 ECOBOOST 95 S/S ST-LINE X 5P,5,1,1,0,7,7,2
2,2025-04-09,TOYOTA,TOYOTA PROACE,COMBI LONG 1.5 D-4D DYNAMIC,23990.0,29.04.2021,71887.0,Diesel,Boite de vitesse manuelle,120,...,Diesel,34290,TOYOTA PROACE 2 II (2) COMBI LONG 1.5 120 D-4D...,6,1,1,0,8,8,2
3,2025-04-09,TOYOTA,TOYOTA PROACE,COMBI LONG 1.5 D-4D DYNAMIC,23990.0,29.04.2021,71887.0,Diesel,Boite de vitesse manuelle,120,...,Diesel,34240,TOYOTA PROACE 2 II (2) PROACE COMBI LONG 1.5 1...,6,1,1,0,8,8,2
4,2025-04-09,MERCEDES,MERCEDES-BENZ GLA,250 E AMG LINE 8G-DCT,32490.0,23.10.2020,59649.0,Hybride,Boite de vitesse automatique,218,...,Hybride,55600,MERCEDES GLA 2 II 250 E AMG LINE 8G-DCT,5,1,1,0,7,7,1


In [12]:
# Changer le format du prix neuf
preprocessor.data['np_prix_neuf'] = preprocessor.data['np_prix_neuf'].astype(float)
# Calculer le prix neuf moyen par annonce avant et après la suppression des outliers (méthode interquartile range)
preprocessor.fix_new_price()

Nombre d'annonces: 2339
Nombre d'annonces avec prix neuf moyen avant suppression des outliers < prix d'occasion: 2
Nombre d'annonces avec prix neuf moyen après suppression des outliers < prix d'occasion: 2


,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,note_carburant_commun,note_nb_porte_commun,note_totale_commune,max_note,nb_match_par_annonce,np_prix_neuf_moy,np_prix_neuf_median,prix_neuf_moyen_iqr,ratio_vr,ratio_vr_iqr
0,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,1,0,7,7,2,21650.0,21650.0,21650.0,0.609238,0.609238
1,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,1,0,7,7,2,21650.0,21650.0,21650.0,0.609238,0.609238
2,2025-04-09,TOYOTA,TOYOTA PROACE,COMBI LONG 1.5 D-4D DYNAMIC,23990.0,29.04.2021,71887.0,Diesel,Boite de vitesse manuelle,120,...,1,0,8,8,2,34265.0,34265.0,34265.0,0.700131,0.700131
3,2025-04-09,TOYOTA,TOYOTA PROACE,COMBI LONG 1.5 D-4D DYNAMIC,23990.0,29.04.2021,71887.0,Diesel,Boite de vitesse manuelle,120,...,1,0,8,8,2,34265.0,34265.0,34265.0,0.700131,0.700131
4,2025-04-09,MERCEDES,MERCEDES-BENZ GLA,250 E AMG LINE 8G-DCT,32490.0,23.10.2020,59649.0,Hybride,Boite de vitesse automatique,218,...,1,0,7,7,1,55600.0,55600.0,55600.0,0.584353,0.584353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6794,2025-04-09,BMW,BMW SÉRIE 1,118I M SPORT DKG7,24990.0,15.12.2020,68713.0,Essence,Boite de vitesse automatique,136,...,1,0,6,6,1,39000.0,39000.0,39000.0,0.640769,0.640769
6795,2025-04-09,BMW,BMW SÉRIE 1,116I M SPORT ULTIMATE,17190.0,29.08.2018,78976.0,Essence,Boite de vitesse manuelle,109,...,1,0,6,6,2,30575.0,30575.0,30575.0,0.562224,0.562224
6796,2025-04-09,BMW,BMW SÉRIE 1,116I M SPORT ULTIMATE,17190.0,29.08.2018,78976.0,Essence,Boite de vitesse manuelle,109,...,1,0,6,6,2,30575.0,30575.0,30575.0,0.562224,0.562224
6797,2025-04-09,RENAULT,RENAULT SCENIC,1.3 TCE SL LIMITED,19990.0,20.04.2021,6321.0,Essence,Boite de vitesse manuelle,140,...,1,0,7,7,2,32300.0,32300.0,32300.0,0.618885,0.618885


Moyennes des prix neufs par annonce avant et après suppression des outliers sont assez identiques  
=> <b> Pour simplicité: Retenir le prix neuf moyen avant suppression des outliers </b>

In [13]:
# Focus sur les annonces où le prix neuf est inférieur au prix de l'annonce
preprocessor.data[(preprocessor.data["np_prix_neuf_moy"] < preprocessor.data["prix"])
                  | (preprocessor.data["prix_neuf_moyen_iqr"] < preprocessor.data["prix"])]

,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,note_carburant_commun,note_nb_porte_commun,note_totale_commune,max_note,nb_match_par_annonce,np_prix_neuf_moy,np_prix_neuf_median,prix_neuf_moyen_iqr,ratio_vr,ratio_vr_iqr
38,2025-04-09,DACIA,DACIA SANDERO,STEPWAY 1.0 TCE,13490.0,24.08.2020,6560.0,Essence,Boite de vitesse manuelle,100,...,1,0,6,6,1,13390.0,13390.0,13390.0,1.007468,1.007468
295,2025-04-09,DACIA,DACIA SANDERO,1.0 TCE EXPRESSION CVT,17790.0,19.04.2023,8092.0,Essence,Boite de vitesse automatique,91,...,1,0,6,6,2,17250.0,17250.0,17250.0,1.031304,1.031304
296,2025-04-09,DACIA,DACIA SANDERO,1.0 TCE EXPRESSION CVT,17790.0,19.04.2023,8092.0,Essence,Boite de vitesse automatique,91,...,1,0,6,6,2,17250.0,17250.0,17250.0,1.031304,1.031304


In [14]:
# 2 annonces où le prix neuf est inférieur au prix de l'annonce
# Les supprimer
preprocessor.data = preprocessor.data[(preprocessor.data["np_prix_neuf_moy"] >= preprocessor.data["prix"])]
preprocessor.data.shape

(6796, 57)

In [15]:
# Renommer la colonne np_prix_neuf_moy en prix_neuf
preprocessor.data.rename(columns={'np_prix_neuf_moy': 'prix_neuf'}, inplace=True)

C:\Users\Hong-CuongLE\AppData\Local\Temp\ipykernel_4752\3084189371.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
print(f"Après la suppression des annonces où le prix neuf est inférieur au prix de l'annonce, le nombre d'annonces restantes est: \
      {preprocessor.data['id_annonce'].drop_duplicates().shape[0]}")

Après la suppression des annonces où le prix neuf est inférieur au prix de l'annonce, le nombre d'annonces restantes est:       2337


### 5. Traitement des valeurs manquantes

In [17]:
# Missing values per column
preprocessor.display_missing_values()

Missing values in each column:
emission_CO2                     645
crit_air                          10
usage_commerciale_anterieure     645
model_alternative               6547
match_type_annee                6547
dtype: int64


#### Emission de CO2

In [18]:
# Create a copy of the DataFrame first
data_copy = preprocessor.data.copy()

# Extract CO2 values and convert to float
data_copy.loc[:, 'co2_caradisiac'] = data_copy['CO2\r\n(g/km)'].str.extract(r'(\d+)').astype(float)

# Calculate mean CO2 per announcement
data_copy.loc[:, 'co2_caradisiac_moy'] = data_copy.groupby('id_annonce')['co2_caradisiac'].transform('mean')

# Fill missing CO2 emissions with calculated mean
data_copy.loc[:, 'emission_CO2'] = data_copy['emission_CO2'].fillna(data_copy['co2_caradisiac_moy'])

# Drop temporary columns
data_copy.drop(columns=['CO2\r\n(g/km)', 'co2_caradisiac', 'co2_caradisiac_moy'], inplace=True)

In [19]:
# Supprimer les doublons d'annonces
data_copy.drop_duplicates(subset=['id_annonce'], inplace=True)
# Liste des colonnes à supprimer
cols_to_drop = ['scraped_at', 'finition', 'annee_mise_en_circulation',
                'url_annonce', 'modele_alt', 'finition_puissance', 'np_url_prix_neuf',
                'modele_annee', 'model_alternative', 'match_type_annee', 'np_marque',
                'np_versions', 'np_model', 'np_version_selected', 'np_nb_porte', 'np_year',
                'np_boite', 'np_energie', 'np_prix_neuf', 'np_version_finale', 'note_version_commune',
                'note_transmission_commune', 'note_carburant_commun', 'note_nb_porte_commun', 'note_totale_commune',
                'max_note', 'nb_match_par_annonce', 'np_prix_neuf_median', 'prix_neuf_moyen_iqr', 'ratio_vr_iqr']
data_copy.drop(columns=cols_to_drop, inplace=True)
print(f"La dimension des données après suppression des doublons et des colonnes inutiles est : {data_copy.shape}")

# Assign back to preprocessor
preprocessor.data = data_copy

# Display missing values
preprocessor.display_missing_values()

La dimension des données après suppression des doublons et des colonnes inutiles est : (2337, 26)
Missing values in each column:
emission_CO2                      1
crit_air                          5
usage_commerciale_anterieure    256
dtype: int64


In [20]:
preprocessor.data[preprocessor.data['emission_CO2'].isnull()]

,marque,modele,prix,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,classe_vehicule,nb_porte,...,usage_commerciale_anterieure,annee,age_days,age_years,age_months,km_per_year,km_per_month,id_annonce,prix_neuf,ratio_vr
4151,RENAULT,RENAULT CLIO,16190.0,42382.0,Hybride,Boite de vitesse automatique,140,2,Citadine,5,...,NaN,2020,1744.0,4.8,58.1,8829.6,729.5,1467,26433.333333,0.612484


Il reste 1 annonce où le taux d'émission de CO2 est manquant  
Récupérer le taux d'émission moyen de la même marque, modele et type de carburant

In [21]:
# Imputer emission_CO2 avec la moyenne de même marque, modèle et carburant
preprocessor.data['emission_CO2'] = preprocessor.data.groupby(['marque', 'modele', 'carburant'])['emission_CO2'].transform(
    lambda x: x.fillna(x.mean())
)

In [22]:
# Re check missing values
preprocessor.display_missing_values()

Missing values in each column:
crit_air                          5
usage_commerciale_anterieure    256
dtype: int64


#### Crit-air

Remplir les valeurs manquantes en se basant sur:  
- Le type de carburant
- Classe d'émission (EURO 5, EURO 6, etc.)

In [23]:
# Distribution des crit-air, y compris les valeurs manquantes
preprocessor.data['crit_air'].value_counts(dropna=False)

crit_air
Crit'Air 1    1742
Crit'Air 2     590
NaN              5
Name: count, dtype: int64

In [24]:
# Focus sur les annonces où crit_air est manquant
df_crit_air_miss = preprocessor.data[preprocessor.data['crit_air'].isnull()]
df_crit_air_miss

,marque,modele,prix,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,classe_vehicule,nb_porte,...,usage_commerciale_anterieure,annee,age_days,age_years,age_months,km_per_year,km_per_month,id_annonce,prix_neuf,ratio_vr
1085,SKODA,SKODA FABIA,15390.0,58873.0,Essence,Boite de vitesse automatique,110,2,Citadine,5,...,Non,2022,1002.0,2.7,33.4,21804.8,1762.7,406,24610.0,0.625356
2520,TOYOTA,TOYOTA YARIS,14890.0,45268.0,Hybride,Boite de vitesse automatique,100,1,Citadine,5,...,Non,2020,1763.0,4.8,58.8,9430.8,769.9,915,21660.5,0.687426
2861,PEUGEOT,PEUGEOT 208,10490.0,32516.0,Essence,Boite de vitesse manuelle,75,2,Citadine,5,...,Non,2022,999.0,2.7,33.3,12043.0,976.5,1025,21420.0,0.489729
5123,OPEL,OPEL CROSSLAND X,17190.0,18281.0,Essence,Boite de vitesse automatique,130,2,SUV,5,...,Oui,2023,799.0,2.2,26.6,8309.5,687.3,1764,30500.0,0.563607
5670,PEUGEOT,PEUGEOT 208,10490.0,32516.0,Essence,Boite de vitesse manuelle,75,2,Citadine,5,...,Non,2022,999.0,2.7,33.3,12043.0,976.5,1965,21420.0,0.489729


In [25]:
# Le crit-air trouvé sur les modèles équivalents
preprocessor.data[(preprocessor.data['modele'].isin(df_crit_air_miss['modele'])) 
                   & (preprocessor.data['carburant'].isin(df_crit_air_miss['carburant']))
                   & (preprocessor.data['crit_air']).notnull()][['modele', 'classe_emission', 'carburant', 'crit_air']].drop_duplicates()

,modele,classe_emission,carburant,crit_air
32,SKODA FABIA,EURO 6,Essence,Crit'Air 1
37,PEUGEOT 208,EURO 6,Essence,Crit'Air 1
40,OPEL CROSSLAND X,EURO 6,Essence,Crit'Air 1
189,TOYOTA YARIS,EURO 6,Hybride,Crit'Air 1
553,TOYOTA YARIS,EURO 6,Essence,Crit'Air 1


In [26]:
# Remplir les valeurs manquantes de crit_air avec le crit'air trouvé sur les modèles équivalents (utiliser mode)
preprocessor.data['crit_air'] = preprocessor.data.groupby(['marque', 'modele', 'carburant'])['crit_air'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
)

In [27]:
# Recheck missing values after filling crit_air
preprocessor.display_missing_values()

Missing values in each column:
usage_commerciale_anterieure    256
dtype: int64


#### Usage commerciale antérieure

In [28]:
preprocessor.data['usage_commerciale_anterieure'].value_counts(dropna=False)

usage_commerciale_anterieure
Non                         1612
Oui                          464
NaN                          256
Oui, Location                  3
Oui, véhicule de société       2
Name: count, dtype: int64

In [29]:
# Si contenir "Oui" => usage_commerciale_anterieure = "Oui"
preprocessor.data.loc[preprocessor.data['usage_commerciale_anterieure'].str.lower().str.contains("oui", na=False), 'usage_commerciale_anterieure'] = "Oui"
# Si contenir "Non" => usage_commerciale_anterieure = "Non"
preprocessor.data.loc[preprocessor.data['usage_commerciale_anterieure'].str.lower().str.contains("non", na=False), 'usage_commerciale_anterieure'] = "Non"
# Si vide ou NaN => usage_commerciale_anterieure = "Unknown"
preprocessor.data.loc[preprocessor.data['usage_commerciale_anterieure'].isnull(), 'usage_commerciale_anterieure'] = "Unknown"

In [30]:
preprocessor.data['usage_commerciale_anterieure'].value_counts(dropna=False)

usage_commerciale_anterieure
Non        1612
Oui         469
Unknown     256
Name: count, dtype: int64

In [31]:
# Recheck missing values after filling crit_air
preprocessor.display_missing_values()

Missing values in each column:
Series([], dtype: int64)


Il n'y a plus de missing value

## III. Echantillonage

Echantillonage en base d'apprentissage et base de test

In [32]:
# Créer une copie du DataFrame pour éviter de modifier l'original
df_preprocessed = preprocessor.data.copy()
# Sauvegarder le DataFrame prétraité
df_preprocessed.to_csv('../data/processed_data/preprocessed_data.csv', index=False)

In [33]:
df_preprocessed.head()

,marque,modele,prix,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,classe_vehicule,nb_porte,...,usage_commerciale_anterieure,annee,age_days,age_years,age_months,km_per_year,km_per_month,id_annonce,prix_neuf,ratio_vr
0,FORD,FORD FIESTA,13190.0,69301.0,Essence,Boite de vitesse manuelle,95,3,Citadine,5,...,Unknown,2020,1790.0,4.9,59.7,14143.1,1160.8,1,21650.0,0.609238
2,TOYOTA,TOYOTA PROACE,23990.0,71887.0,Diesel,Boite de vitesse manuelle,120,2,Monospace,4,...,Oui,2021,1441.0,3.9,48.0,18432.6,1497.6,2,34265.0,0.700131
4,MERCEDES,MERCEDES-BENZ GLA,32490.0,59649.0,Hybride,Boite de vitesse automatique,218,2,SUV,5,...,Oui,2020,1629.0,4.5,54.3,13255.3,1098.5,3,55600.0,0.584353
5,BMW,BMW X1,27890.0,37869.0,Essence,Boite de vitesse automatique,136,2,SUV,5,...,Non,2021,1435.0,3.9,47.8,9710.0,792.2,4,41850.0,0.666428
6,PEUGEOT,PEUGEOT 3008,19090.0,58958.0,Diesel,Boite de vitesse automatique,130,3,SUV,5,...,Oui,2019,1926.0,5.3,64.2,11124.2,918.3,5,38650.0,0.493920


In [34]:
# Définition de la variable cible
target_variable = 'ratio_vr'

In [35]:
# Tout d'abord, faire un split des données en train et test
X = df_preprocessed.drop(columns=[target_variable])
y = df_preprocessed[target_variable]

In [ ]:
# Echantillonnage stratifié en fonction de l'année pour éviter le déséquilibre
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42, stratify=X['annee'])

In [ ]:
# Vérifier la répartition par année dans X_train et X_test
print("Répartition par année dans X_train:")
print((X_train['annee'].value_counts()/len(X_train)).sort_index())

print("Répartition par année dans X_test:")
print((X_test['annee'].value_counts()/len(X_test)).sort_index())

In [ ]:
# Vérifier le nombre d'observations par échantillon
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train_modele = np.unique(X_train['modele'])

In [ ]:
X_test_modele = np.unique(X_test['modele'])

In [ ]:
unseen_X_train = [mod for mod in X_test_modele if mod not in X_train_modele]

In [ ]:
unseen_X_train

## IV. Explanatory Data Analyses (Train set)

### 1. Analyses univariées

#### a. Variables catégorielles

In [ ]:
# Liste des variables catégorielles
categorical_variables = X_train.select_dtypes(include=['object']).columns.tolist()

In [ ]:
univariate_analysis(X_train, list_columns=categorical_variables, dtype="cat")

<u><b> Conclusion </b></u>:  
N.B: "Autres" = regroupement des modalités qui ont moins de 30 observations, quand il y a plus de 10 catégories
- Marque: Il y a 34 marques différentes dans le dataset. La majorité contient des marques françaises: Peugeot, Renault  
- Modèle: 221 modèles différents toutes marques comprises. Pas forcément un certain modèle qui domine la répartition
- Carburant:  Il y a 4 types de carburant recensés dans le dataset. La grande majorité est "Essence" avec 67% des obs, suivie par "Diesel" (25%). Hybride et Ethanol représentent peu. Pas Electrique dans le dataset.  
- Boite de vitesse: assez équilibré entre Automatique et Manuelle, avec des fréquences relativement plus hautes en boite Automatique  
- Nombre d'ancien propriétaire: 1 ou 2 anciens propriétaies en général  
- Classe véhicule: la majorité est SUV, Citadine, Berline  
- Nombre de porte & nombre de place: presque à 5 portes & 5 places  => assez traditionnel  
- Couleur: Gris, Blanc et Noir sont majoritaires
- Sellerie: Tissu ou Mi-cuir en général
- Classe d'émisison: presque la totalité est EURO-6
- Critair: la grande majorité est en Critair 1, le reste en Critair 2  => Voitures relativement au norme  
- Usage commerciale antérieure: Non dans 70% des cas
- Année de mise en circulation: la majorité est entre 2018 et 2021, très peu de fréquence sur les années récentes (2023, 2024)

#### b. Variables continues

In [ ]:
# Liste des variables continues, y compris la variable cible
continuous_variables = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Supprimer des variables qui ne sont pas pertinentes pour l'analyse univariée
to_drop = ['id_annonce']
continuous_variables = [col for col in continuous_variables if col not in to_drop]

print(f"Liste des variables continues: {continuous_variables}")

In [ ]:
univariate_analysis(X_train, list_columns=continuous_variables, dtype="num")

La distribution de prix d'occasion est relativement asymétrique à droite, avec une moyenne autour de 17K€  
Par construction lors de la collecte des données, le kilométrage est borné à 100K km. Année de mise en circulation est floorée à 2017   
Le ratio VR moyen est aux alentours de 58% et celui-ci est proche de la médiane

### 2. Target vs variables catégorielles

In [ ]:
# Rassembler X_train et y_train pour l'analyse bivariée
X_y_train = pd.concat([X_train, y_train], axis=1) 
bivariate_analysis(X_y_train, list_columns=categorical_variables, dtype="cat", target_column=target_variable)

Les marques et modèles sont nombreux => Besoin potentiel de les regrouper ?  
Le taux de VR est moyennement plus élevé sur les voitures d'essence que diesel  
Il n'y a pas forcément d'écart significatif de ratio VR moyen entre les voitures de boite auto et de boite manuelle dans cet échantillon  
Les voitures qui ont 1 ancien propriétaire uniquement se vendent généralement plus cher  

In [ ]:
univariate_analysis(X_y_train, list_columns=categorical_variables, dtype="cat", target_column=target_variable)

La répartition des modalités au sein des variables comme: nb_place, nb_porte, classe_emission 
n'est pas équilibre. Forte concentration sur une modalité

In [ ]:
# Regrouper les types de sellerie, ne prendre que la matière = élément avant la parenthèse
X_train['sellerie'] = X_train['sellerie'].str.split('(').str[0].str.strip()
print(f"Liste de sellerie après le regroupement: {sorted(set(X_train['sellerie']))}")

# Appliquer pour X_test
X_test['sellerie'] = X_test['sellerie'].str.split('(').str[0].str.strip()
print(f"Liste de sellerie après le regroupement pour l'échantillon de test: {sorted(set(X_test['sellerie']))}")

# Appliquer pour X_y_train
X_y_train['sellerie'] = X_y_train['sellerie'].str.split('(').str[0].str.strip()
print(f"Liste de sellerie après le regroupement pour l'échantillon de XY train: {sorted(set(X_y_train['sellerie']))}")

### 3. Target vs variables continues

In [ ]:
num_var_focus = ['prix_neuf', 'age_months', 'kilometrage', 'puissance', 'emission_CO2']
g = sns.PairGrid(X_y_train, vars=num_var_focus)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.fig_size = (8, 12)

In [ ]:
top_modele = X_y_train["modele"].value_counts().nlargest(5).index.tolist()
print(f"Top 5 modèles: {top_modele}")

for i in top_modele:
    data = X_y_train[(X_y_train["modele"]==i)]
    plt.subplots(figsize=(10, 6))
    sns.lineplot(x=data['age_months'], y=data[target_variable], label=i)

In [ ]:
bivariate_analysis(X_y_train, list_columns=continuous_variables, dtype="num", target_column=target_variable)

### 4. Analyses multivariées

In [ ]:
# VR par l'âge du véhicule et classe de véhicule
multivariate_analysis(X_y_train, x="age_months", y = target_variable, 
                      row="classe_vehicule", col="transmission", hue="carburant")

In [ ]:
# Dynamique de VR en fonction de la marque, carburant et age
# Focus sur les marques qui ont plus de 30 annonces
X_y_train['marque_count'] = X_y_train.groupby('marque')['id_annonce'].transform('count')
X_y_train_30 = X_y_train[X_y_train['marque_count'] > 30]
multivariate_analysis(X_y_train_30, x="age_months", y = target_variable, 
                      row="marque", col="carburant")

In [ ]:
# Graphique de 3 dimensions avec Plotly: ratio_vr, age_months et kilometrage
scatter_3d(X_y_train, x="age_months", y="kilometrage", z=target_variable,
           x_title ="Age (mois)", y_title="Kilometrage (km)", z_title="Ratio VR", legend_title="Carburant", 
           color="carburant")

En général, Ratio VR en baisse avec l'augmentation de kilométrage parcouru et de l'âge du véhicule  
- Les véhicules Diesel sont plutôt anciens et de haut kilométrage.  
- A rappeler néanmoins la forte répartition des véhicules d'essence dans ce jeu de données

In [ ]:
# Age et kilométrage moyen par type de carburant
agg_df_carb = X_y_train.groupby("carburant").agg(nb_annonce = ("id_annonce", "count"),
                                                       age_month_moyen = ("age_months", "mean"),
                                                       km_moyen = ("kilometrage", "mean"),
                                                       vr_moyen = ("ratio_vr", "mean")
                                                     )
agg_df_carb.insert(agg_df_carb.columns.get_loc("nb_annonce") + 1, 
                   "nb_annonce_pct", 
                   (agg_df_carb['nb_annonce']/agg_df_carb["nb_annonce"].sum()))
agg_df_carb.sort_values(by="nb_annonce_pct", ascending=False)

In [ ]:
# Graphique de 3 dimensions avec Plotly: ratio_vr, age_months et prix_neuf
scatter_3d(X_y_train, x="age_months", y="prix_neuf", z=target_variable,
           x_title ="Age (mois)", y_title="Prix neuf", z_title="Ratio VR", legend_title="Carburant", 
           color="carburant")

In [ ]:
# Graphique de 3 dimensions avec Plotly: ratio_vr, kilometrage et prix_neuf
scatter_3d(X_y_train, x="kilometrage", y="prix_neuf", z=target_variable,
           x_title ="Kilometrage (km)", y_title="Prix neuf", z_title="Ratio VR", legend_title="Carburant", 
           color="carburant")

## V. Corrélation

In [ ]:
X_y_train_corr = X_y_train.drop(columns=["id_annonce", "annee", "marque_count", "prix"])
cat_var_xy = X_y_train_corr.select_dtypes(include=['object']).columns.tolist()

In [ ]:
# Création des dummies
df_X_y_train = pd.get_dummies(X_y_train_corr, columns = cat_var_xy, drop_first=True)

In [ ]:
corr_spearman = df_X_y_train.corr(method="spearman")
corr_target = corr_spearman[target_variable]
df_corr = pd.DataFrame(corr_target)
df_corr.sort_values(by=target_variable, key=lambda x: x.abs(), ascending=False, inplace=True)
df_corr

- L'age et le kilométrage parcouru font partie des variables les plus discriminantes p/r au ratio VR
- La marque Peugeot a une décote plus significative que d'autres marques dans l'échantillon  
- Les véhicules de marque Dacia ont tendance de garder sa valeur dans le temps  

In [ ]:
cmap = sns.diverging_palette(125, 28, s=100, l=65, sep=50, as_cmap=True)
fig, ax = plt.subplots(figsize=(9, 8), dpi=80)
ax = sns.heatmap(df_X_y_train[['kilometrage', 'puissance', 'emission_CO2', 'age_months',
                               'km_per_month', 'prix_neuf', target_variable]].corr(method="spearman"), annot=True, cmap = cmap)
plt.show()

## VI. Categorical Embedding

In [ ]:
# Nombre d'année group by modèle
X_train["modele"].value_counts()

In [ ]:
X_y_train.groupby("modele").agg(nb_annonce = ("id_annonce", "count"),
                                nb_annee = ("annee", "nunique"),
                                age_moyen = ("age_years", "mean"),   
                                km_moyen = ("kilometrage", "mean"),
                                vr_moyen = ("ratio_vr", "mean")).sort_values(by="nb_annonce", ascending=False)

In [ ]:
X_train["marque"].value_counts()

In [ ]:
X_y_train.groupby("marque").agg(nb_annonce = ("id_annonce", "count"),
                                nb_annee = ("annee", "nunique"),
                                age_moyen = ("age_years", "mean"),   
                                km_moyen = ("kilometrage", "mean"),
                                vr_moyen = ("ratio_vr", "mean")).sort_values(by="nb_annonce", ascending=False)

Il serait intéressant de regrouper les modèles homogènes et pareillement pour les marques:  
- Réduire la dimension: on a 217 modèles différents dans le dataset => Beaucoup de modalités en one hot encoding alors que les données ne sont pas forcément représentatives sur qq uns. En plus, one-hot encoding ne permet pas de voir la similarité entre les différents modèles  
- Gérer les nouveaux modèles de voiture qui ne sont pas encore dans le dataset actuel

In [ ]:
embed_features = ['marque', 'modele']
categorical_features = ['carburant', 'transmission', 'classe_vehicule', 'couleur']
numerical_features = ['kilometrage', 'puissance', 'emission_CO2', 'age_months', 'prix_neuf']

# Limiter à la dimension de 2 pour les embeddings pour une meilleure visualisation
embedding_dims = {'marque': 2, 'modele': 2}

# Create model instance
model_handler = CategoricalEmbedding(
        df = X_y_train,
        embed_features=embed_features,
        categorical_features=categorical_features,
        numerical_features=numerical_features,
        embedding_dims=embedding_dims,
        target_column=target_variable,
        hidden_layers = [128],
        dropout_rates= [0.1]
    )
# Prepare data for training
X_train_array, y_train_array = model_handler.prepare_data()

In [ ]:
model = model_handler.create_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics =['mae'])

In [ ]:
# Train the model
history = model.fit(X_train_array, y_train_array,  
                    epochs=10, batch_size=32)

In [ ]:
# Summary history for Loss
plt.plot(history.history['loss'], label ='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Summary history for MAE
plt.plot(history.history['mae'], label ='Training MAE')
plt.title('Training MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
visualize_all_embeddings(model_handler, model)

In [ ]:
y_pred = model.predict(X_train_array)
y_pred = y_pred.flatten()
ax = sns.scatterplot(x=y_train_array, y=y_pred)

model_error = y_train_array - y_pred
R2 = 1 - sum(model_error**2) / sum((y_train_array - np.mean(y_train_array))**2)
RMSE = np.sqrt(np.mean(model_error**2))
MAE = np.mean(np.abs(model_error))
print(f"R2: {R2:.4f}, RMSE: {RMSE:.4f}, MAE: {MAE:.4f}")

In [ ]:
df_embed_marque = get_embedding_weights(model, model_handler.encoders['marque'], 'marque')
df_embed_marque.head()

In [ ]:
df_embed_modele = get_embedding_weights(model, model_handler.encoders['modele'], 'modele')
df_embed_modele.head()

In [ ]:
# df_embed_marque_clustered = cluster_embeddings(df_embed_marque, "marque", n_clusters=4, list_columns=['marque_0', 'marque_1'])

In [ ]:
# df_embed_marque_clustered

In [ ]:
# df_embed_modele_clustered = cluster_embeddings(df_embed_modele, "modele", n_clusters=5, list_columns=['modele_0', 'modele_1'])

In [ ]:
# df_embed_modele_clustered

## VII. Variables encoding and scaling pipelines

In [ ]:
# Nombre de modalités par variable catégorielle
{col : X_train[col].nunique() for col in X_train.select_dtypes(include="object").columns}

In [ ]:
X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
# Ajouter les coordonnées par marque dans les DataFrames d'entraînement et de test
df_embed_marque.reset_index(inplace=True)
df_embed_marque.rename(columns={'index': 'marque'}, inplace=True)
X_train_upd = pd.merge(X_train, df_embed_marque, on="marque", how="left")
X_y_train_upd = pd.merge(X_y_train, df_embed_marque, on="marque", how="left")
X_test_upd = pd.merge(X_test, df_embed_marque, on="marque", how="left")

# Ajouter les coordonnées par modèle dans les DataFrames d'entraînement et de test
df_embed_modele.reset_index(inplace=True)
df_embed_modele.rename(columns={'index': 'modele'}, inplace=True)
X_train_upd = pd.merge(X_train_upd, df_embed_modele, on="modele", how="left")
X_y_train_upd = pd.merge(X_y_train_upd, df_embed_modele, on="modele", how="left")
X_test_upd = pd.merge(X_test_upd, df_embed_modele, on="modele", how="left")

In [ ]:
# Calculer la moyenne des points dans X_train pour utiliser dans les pipelines de transformation
imputer_marque_0 = df_embed_marque["marque_0"].mean()
imputer_marque_1 = df_embed_marque["marque_1"].mean()
print(f"Imputation pour marque: \n imputer_marque_0: {imputer_marque_0: .4f}, imputer_marque_1: {imputer_marque_1: .4f}")

imputer_modele_0 = df_embed_modele["modele_0"].mean()
imputer_modele_1 = df_embed_modele["modele_1"].mean()
print(f"Imputation pour modele: \n imputer_modele_0: {imputer_modele_0: .4f}, imputer_modele_1: {imputer_modele_1: .4f}")

In [ ]:
# Supprimer les colonnes non pertinentes pour l'analyse
# id_annonce: c'est un identifiant unique pour chaque annonce, donc pas pertinent pour l'analyse
# annee: c'est l'année de l'annonce, mais on a déjà l'âge de la voiture en mois, donc pas nécessaire
# marque et modèle à remplacer par les embeddings
# La répartition des modalités au sein des variables comme: nb_place, nb_porte, classe_emission 
# n'est pas équilibre. Forte concentration sur une modalité => Supprimer ces variables pour éviter de biaiser le modèle.
# prix: c'est utilisé pour calculer la variable cible (ratio_vr), donc pas nécessaire dans les DataFrames d'entraînement et de test
# On va donc supprimer ces deux colonnes des DataFrames d'entraînement et de test
X_train_upd.drop(columns=['id_annonce', 'annee', 'modele', 'marque', 'prix', 'age_years', 'age_days', 'km_per_year', 'nb_place', 'nb_porte', 'classe_emission'], inplace=True)
X_y_train_upd.drop(columns=['id_annonce', 'annee', 'modele', 'marque', 'prix', 'age_years', 'age_days', 'km_per_year', 'nb_place', 'nb_porte', 'classe_emission'], inplace=True)
X_test_upd.drop(columns=['id_annonce', 'annee', 'modele', 'marque', 'prix', 'age_years', 'age_days', 'km_per_year', 'nb_place', 'nb_porte', 'classe_emission'], inplace=True)

In [ ]:
cat_features_update = X_train_upd.select_dtypes(include=['object']).columns.tolist()
cat_to_transform = [cat for cat in cat_features_update if cat not in ['nb_ancien_proprietaire']]
print(cat_to_transform)

In [ ]:
# Valeur spéciale à imputer pour les variables catégorielles 
special_cat = {'nb_ancien_proprietaire': '1'}

In [ ]:
num_features_update = X_train_upd.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Regular num
num_to_transform = [num for num in num_features_update if num not in ['marque_0', 'marque_1', 'modele_0', 'modele_1']]
print(num_to_transform)

In [ ]:
# Variables numériques et leurs imputations spéciales
special_num = {'marque_0' : imputer_marque_0,
               'marque_1' : imputer_marque_1,
               'modele_0' : imputer_modele_0,
               'modele_1' : imputer_modele_1}
print(special_num)

In [ ]:
# Pipeline pour encoder les variables catégorielles & numériques
transform_pipeline = create_transformer_pipeline(categorical_features=cat_to_transform,
                                                 special_impute_cat_features=special_cat,
                                                 numerical_features=num_to_transform,
                                                 special_impute_num_features=special_num)
transform_pipeline

In [ ]:
# Appliquer pipeline de transformation pour X_train et X_test
X_train_prepared = transform_pipeline.fit_transform(X_train_upd)
X_test_prepared = transform_pipeline.transform(X_test_upd)

In [ ]:
X_train_prepared.shape, X_test_prepared.shape

In [ ]:
# Display all transformers in the pipeline
for name, transformer, features in transform_pipeline.transformers_:
	print(f"Name: {name}, Transformer: {transformer}, Features: {features}")
	# Only access 'onehot' if it exists in the pipeline steps
	if hasattr(transformer, 'named_steps') and 'onehot' in transformer.named_steps:
		print(transformer.named_steps['onehot'])

In [ ]:
# Ajouter le nom des colonnes dans la data transformée
feature_names = []
for name, transformer, features in transform_pipeline.transformers_:
    if name == 'remainder':
        feature_names.extend(X_train.columns[features])
    if name == 'regular_cat':
        encoder_cat = transformer.named_steps['onehot']
        feature_names.extend(encoder_cat.get_feature_names_out(features))
    elif name in special_cat.keys():
        encoder_spe_cat = transformer.named_steps['onehot']
        feature_names.extend(encoder_spe_cat.get_feature_names_out(features))
    elif name == 'regular_num':
        feature_names.extend(features)
    elif name in special_num.keys():
        feature_names.extend(features)

X_train_transformed_df = pd.DataFrame(X_train_prepared, columns=feature_names) 
X_test_transformed_df = pd.DataFrame(X_test_prepared, columns=feature_names) 

In [ ]:
X_train_transformed_df

## VIII. Training

### Linear Regression - Baseline model

In [ ]:
# Lancer une régression linéaire pour prédire le ratio VR
lin_reg = LinearRegression()
lin_reg.fit(X_train_prepared, y_train)

In [ ]:
pd.DataFrame(data=np.append(lin_reg.intercept_, lin_reg.coef_),
             index = ['Intercept'] + [col for col in X_train_transformed_df.columns],
             columns = ['Value']
             ).sort_values(by='Value', ascending =False)

In [ ]:
lin_reg_eval_train = model_evaluation(lin_reg, X_train_prepared, y_train, "Linear Regression Train")

In [ ]:
lin_reg_eval = model_evaluation(lin_reg, X_test_prepared, y_test, "Linear Regression Test")

### XG Boost

In [ ]:
# Tester XG Boost
xg_boost = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state= 42, max_depth=5)
xg_boost.fit(X_train_transformed_df, y_train)

In [ ]:
xg_boost_eval_train = model_evaluation(xg_boost, X_train_transformed_df, y_train, "XG Boost Regression Train")

In [ ]:
xg_boost_eval = model_evaluation(xg_boost, X_test_transformed_df, y_test, "XG Boost Regression Test")

In [ ]:
# Feature importance for XG Boost
xg_boost_importance = xg_boost.get_booster().get_score(importance_type='weight')
xg_boost_importance_df = pd.DataFrame(xg_boost_importance.items(), columns=['Feature', 'Importance'])
xg_boost_importance_df.sort_values(by='Importance', ascending=False, inplace=True)
xg_boost_importance_df.reset_index(drop=True, inplace=True)
xg_boost_importance_df.nlargest(10, columns="Importance").plot(kind='barh', x='Feature', y='Importance', figsize=(10,6), title = "Top 10 Features Importance for XG Boost Regression")

In [ ]:
# Visualiser y_train_pred et age_months et kilometrage
y_train_pred = xg_boost.predict(X_train_prepared)
# Ajouter les prédictions au DataFrame d'entraînement
X_y_train_upd[target_variable + '_pred_XG_boost'] = y_train_pred

scatter_3d(X_y_train_upd, x="age_months", y="kilometrage", z=target_variable + '_pred_XG_boost',
           x_title ="Age (mois)", y_title="Kilometrage (km)", z_title="Ratio VR Prédit")

### XG Boost + monotonicity constrained

In [ ]:
# Ajouter les contraintes de monotonie
# -1 pour une contrainte décroissante par rapport à la VR ratio
monotonic_constraints = {"kilometrage": -1, "age_months": -1, 'prix_neuf': -1}
xg_boost_cst = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state= 42, max_depth = 5, monotone_constraints=monotonic_constraints)
xg_boost_cst.fit(X_train_transformed_df, y_train)

In [ ]:
xg_boost_cst_eval_train = model_evaluation(xg_boost_cst, X_train_transformed_df, y_train, "XG Boost Regression + Monotonic Constraints on Train set")

In [ ]:
# Feature importance for XG Boost
xg_boost_importance = xg_boost_cst.get_booster().get_score(importance_type='weight')
xg_boost_importance_df = pd.DataFrame(xg_boost_importance.items(), columns=['Feature', 'Importance'])
xg_boost_importance_df.sort_values(by='Importance', ascending=False, inplace=True)
xg_boost_importance_df.reset_index(drop=True, inplace=True)
xg_boost_importance_df.nlargest(10, columns="Importance").plot(kind='barh', x='Feature', y='Importance', figsize=(10,6), title = "Top 10 Features Importance for XG Boost Regression")

In [ ]:
xg_boost_cst_eval = model_evaluation(xg_boost_cst, X_test_transformed_df, y_test, "XG Boost Regression + Monotonic Constraints on Test set")

In [ ]:
# Visualiser y_train_pred et age_months et kilometrage
y_train_pred = xg_boost_cst.predict(X_train_prepared)
# Ajouter les prédictions au DataFrame d'entraînement
X_y_train_upd[target_variable + '_pred_XG_boost_cst'] = y_train_pred

scatter_3d(X_y_train_upd, x="age_months", y="kilometrage", z=target_variable + '_pred_XG_boost_cst',
           x_title ="Age (mois)", y_title="Kilometrage", z_title="Ratio VR Prédit")

### XG Boost + monotonicity constrained + Regularization Lasso

In [ ]:
# Ajouter les contraintes de monotonie
# -1 pour une contrainte décroissante par rapport à la VR ratio
monotonic_constraints = {"kilometrage": -1, "age_months": -1, 'prix_neuf': -1}

In [ ]:
# Créer une boucle pour évaluer le modèle avec différentes valeurs de alpha
alpha = [0.01, 0.1, 0.5, 1, 2, 5, 10]

df_eval_xgboost_train = pd.DataFrame()
df_eval_xgboost_test = pd.DataFrame()

for a in alpha:
    xg_boost_cst_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state= 42, max_depth = 5, 
                                        alpha = a, 
                                        monotone_constraints=monotonic_constraints)
    xg_boost_cst_reg.fit(X_train_transformed_df, y_train)

    print(f"Evaluation for alpha = {a}")
    
    xg_boost_cst_reg_eval_train = model_evaluation(xg_boost_cst_reg, X_train_transformed_df, y_train, f"Alpha {a} on Train set")
    xg_boost_cst_reg_eval = model_evaluation(xg_boost_cst_reg, X_test_transformed_df, y_test, f"Alpha {a} on Test set")

    df_eval_xgboost_train = pd.concat([df_eval_xgboost_train, xg_boost_cst_reg_eval_train], axis=1)
    df_eval_xgboost_test = pd.concat([df_eval_xgboost_test, xg_boost_cst_reg_eval], axis=1)


In [ ]:
df_eval_xgboost_train

In [ ]:
df_eval_xgboost_test

In [ ]:
# Keep alpha = 0.5
xg_boost_cst_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state= 42, max_depth = 5, 
                                        alpha = 0.5, 
                                        monotone_constraints=monotonic_constraints)
xg_boost_cst_reg.fit(X_train_transformed_df, y_train)

# Feature importance for XG Boost
xg_boost_importance = xg_boost_cst_reg.get_booster().get_score(importance_type='weight')
xg_boost_importance_df = pd.DataFrame(xg_boost_importance.items(), columns=['Feature', 'Importance'])
xg_boost_importance_df.sort_values(by='Importance', ascending=False, inplace=True)
xg_boost_importance_df.reset_index(drop=True, inplace=True)
xg_boost_importance_df.nlargest(10, columns="Importance").plot(kind='barh', x='Feature', y='Importance', figsize=(10,6), title = "Top 10 Features Importance for XG Boost Regression")

test_evaluation = model_evaluation(xg_boost_cst_reg, X_test_transformed_df, y_test, "XG Boost Regression + Monotonic Constraints on Test set with alpha = 1")

## IX. Model saving

In [ ]:
df_embed_marque

In [ ]:
# Création d'un répertoire pour sauvegarder les modèles
model_dir = os.path.join('..', 'models')
os.makedirs(model_dir, exist_ok=True)

# Sauvegarder le modèle XG Boost avec les contraintes de monotonie + régularisation
model_path = os.path.join(model_dir, 'xg_boost_cst_reg.joblib')
joblib.dump(xg_boost_cst_reg, model_path)

# Sauvegarder le pipeline de transformation
transformer_path = os.path.join(model_dir, 'transform_pipeline.joblib')
joblib.dump(transform_pipeline, transformer_path)

# Sauvegarder le df pour embedding marque et modele
embed_marque = os.path.join(model_dir, 'embedding_marque.joblib')
joblib.dump(df_embed_marque, embed_marque)
embed_modele = os.path.join(model_dir, 'embedding_modele.joblib')
joblib.dump(df_embed_modele, embed_modele)

In [36]:
X.head()

,marque,modele,prix,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,classe_vehicule,nb_porte,...,crit_air,usage_commerciale_anterieure,annee,age_days,age_years,age_months,km_per_year,km_per_month,id_annonce,prix_neuf
0,FORD,FORD FIESTA,13190.0,69301.0,Essence,Boite de vitesse manuelle,95,3,Citadine,5,...,Crit'Air 1,Unknown,2020,1790.0,4.9,59.7,14143.1,1160.8,1,21650.0
2,TOYOTA,TOYOTA PROACE,23990.0,71887.0,Diesel,Boite de vitesse manuelle,120,2,Monospace,4,...,Crit'Air 2,Oui,2021,1441.0,3.9,48.0,18432.6,1497.6,2,34265.0
4,MERCEDES,MERCEDES-BENZ GLA,32490.0,59649.0,Hybride,Boite de vitesse automatique,218,2,SUV,5,...,Crit'Air 1,Oui,2020,1629.0,4.5,54.3,13255.3,1098.5,3,55600.0
5,BMW,BMW X1,27890.0,37869.0,Essence,Boite de vitesse automatique,136,2,SUV,5,...,Crit'Air 1,Non,2021,1435.0,3.9,47.8,9710.0,792.2,4,41850.0
6,PEUGEOT,PEUGEOT 3008,19090.0,58958.0,Diesel,Boite de vitesse automatique,130,3,SUV,5,...,Crit'Air 2,Oui,2019,1926.0,5.3,64.2,11124.2,918.3,5,38650.0


In [37]:
# Save some data
car_data = X.drop(columns=['id_annonce', 'prix', 'age_years', 'age_days', 'km_per_month', 'km_per_year', 'nb_place', 'nb_porte', 'classe_emission'])

In [38]:
car_data.head()

,marque,modele,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,classe_vehicule,couleur,sellerie,emission_CO2,crit_air,usage_commerciale_anterieure,annee,age_months,prix_neuf
0,FORD,FORD FIESTA,69301.0,Essence,Boite de vitesse manuelle,95,3,Citadine,Gris,Tissu (Sellerie d'origine),117.0,Crit'Air 1,Unknown,2020,59.7,21650.0
2,TOYOTA,TOYOTA PROACE,71887.0,Diesel,Boite de vitesse manuelle,120,2,Monospace,Gris,Tissu (Sellerie d'origine),170.0,Crit'Air 2,Oui,2021,48.0,34265.0
4,MERCEDES,MERCEDES-BENZ GLA,59649.0,Hybride,Boite de vitesse automatique,218,2,SUV,Gris,Mi-cuir (Sellerie d'origine),32.0,Crit'Air 1,Oui,2020,54.3,55600.0
5,BMW,BMW X1,37869.0,Essence,Boite de vitesse automatique,136,2,SUV,Noir,Mi-cuir (Sellerie d'origine),148.0,Crit'Air 1,Non,2021,47.8,41850.0
6,PEUGEOT,PEUGEOT 3008,58958.0,Diesel,Boite de vitesse automatique,130,3,SUV,Blanc,Mi-cuir (Sellerie d'origine),98.0,Crit'Air 2,Oui,2019,64.2,38650.0


In [39]:
# Date de mise en circulation = 09/04/2025 - age_months
car_data['mise_en_circulation'] = car_data['age_months'].apply(
	lambda m: pd.to_datetime('2025-04-09') - pd.DateOffset(months=int(m)) if pd.notnull(m) else pd.NaT
)
car_data['mise_en_circulation'] = car_data['mise_en_circulation'].dt.strftime('%d/%m/%Y')
car_data['sellerie'] = car_data['sellerie'].str.split('(').str[0]

# Créer une date de fin du contrat aléatoire (entre 3 et 5 ans après la date de mise en circulation)
car_data['fin_du_contrat'] = car_data['mise_en_circulation'].apply(
    lambda x: pd.to_datetime(x, format='%d/%m/%Y') + pd.DateOffset(years=np.random.randint(3,6)) if pd.notnull(x) else pd.NaT
).dt.strftime('%d/%m/%Y')

car_data.drop("age_months", axis=1, inplace=True)

In [40]:
car_data

,marque,modele,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,classe_vehicule,couleur,sellerie,emission_CO2,crit_air,usage_commerciale_anterieure,annee,prix_neuf,mise_en_circulation,fin_du_contrat
0,FORD,FORD FIESTA,69301.0,Essence,Boite de vitesse manuelle,95,3,Citadine,Gris,Tissu,117.0,Crit'Air 1,Unknown,2020,21650.000000,09/05/2020,09/05/2025
2,TOYOTA,TOYOTA PROACE,71887.0,Diesel,Boite de vitesse manuelle,120,2,Monospace,Gris,Tissu,170.0,Crit'Air 2,Oui,2021,34265.000000,09/04/2021,09/04/2024
4,MERCEDES,MERCEDES-BENZ GLA,59649.0,Hybride,Boite de vitesse automatique,218,2,SUV,Gris,Mi-cuir,32.0,Crit'Air 1,Oui,2020,55600.000000,09/10/2020,09/10/2025
5,BMW,BMW X1,37869.0,Essence,Boite de vitesse automatique,136,2,SUV,Noir,Mi-cuir,148.0,Crit'Air 1,Non,2021,41850.000000,09/05/2021,09/05/2024
6,PEUGEOT,PEUGEOT 3008,58958.0,Diesel,Boite de vitesse automatique,130,3,SUV,Blanc,Mi-cuir,98.0,Crit'Air 2,Oui,2019,38650.000000,09/12/2019,09/12/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6776,VOLKSWAGEN,VOLKSWAGEN T-ROC,18387.0,Essence,Boite de vitesse automatique,150,Inconnu,SUV,Blanc,Tissu,148.0,Crit'Air 1,Non,2021,38445.882353,09/02/2021,09/02/2026
6793,RENAULT,RENAULT TWINGO,36364.0,Essence,Boite de vitesse automatique,92,1,Citadine,Blanc,Mi-cuir,129.0,Crit'Air 1,Non,2020,17600.000000,09/10/2020,09/10/2023
6794,BMW,BMW SÉRIE 1,68713.0,Essence,Boite de vitesse automatique,136,Inconnu,Berline,Blanc,Alcantara,134.0,Crit'Air 1,Non,2020,39000.000000,09/12/2020,09/12/2025
6795,BMW,BMW SÉRIE 1,78976.0,Essence,Boite de vitesse manuelle,109,2,Berline,Blanc,Cuir,134.0,Crit'Air 1,Unknown,2018,30575.000000,09/08/2018,09/08/2023


In [41]:
# Créer un répertoire pour sauvegarder les données à utiliser dans l'outil app
outil_data_dir= os.path.join("..", "data", "outil_data")
os.makedirs(outil_data_dir, exist_ok=True)

# Sauvegarder les données 
car_data_path = os.path.join(outil_data_dir, 'sample_app_car_data.csv')
car_data.to_csv(car_data_path, index=False)